In [41]:
import pandas as pd
balanced_labels = pd.read_csv('../all_models_combine/chosen_labels_all_rules.csv', sep=';',index_col=False)
predictions_labels = pd.read_csv("../all_models_combine/predicted_labels_not_all_rules_3.csv", sep=";", index_col=False)

In [42]:
balanced_labels.describe()

,Rule1,Rule2,Rule3,Rule4,Rule5,Rule6,Rule7,Rule8
count,840.000000,840.000000,840.000000,840.000000,840.000000,840.000000,840.000000,840.000000
mean,0.255952,0.266667,0.135714,0.115476,0.280952,0.498810,0.140476,0.119048
std,0.436655,0.442480,0.342689,0.319786,0.449732,0.500296,0.347687,0.324037
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
75%,1.000000,1.000000,0.000000,0.000000,1.000000,1.000000,0.000000,0.000000
max,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000


In [43]:
df_filtered = balanced_labels[balanced_labels['Image_name'].isin(predictions_labels['Image_name'])].drop_duplicates(subset=['Image_name'],keep="first")
predictions_labels=predictions_labels.drop_duplicates(subset=['Image_name'],keep="first")

In [44]:
predictions_labels.describe()

,Rule1,Rule2,Rule5,Rule6,Rule7
count,754.000000,754.000000,754.000000,754.000000,754.000000
mean,0.187003,0.224138,0.290451,0.501326,0.157825
std,0.390172,0.417290,0.454272,0.500330,0.364819
min,0.000000,0.000000,0.000000,0.000000,0.000000
25%,0.000000,0.000000,0.000000,0.000000,0.000000
50%,0.000000,0.000000,0.000000,1.000000,0.000000
75%,0.000000,0.000000,1.000000,1.000000,0.000000
max,1.000000,1.000000,1.000000,1.000000,1.000000


In [45]:
df_filtered.describe()

,Rule1,Rule2,Rule3,Rule4,Rule5,Rule6,Rule7,Rule8
count,754.000000,754.000000,754.000000,754.000000,754.000000,754.000000,754.000000,754.000000
mean,0.188329,0.198939,0.055703,0.033156,0.200265,0.442971,0.059682,0.037135
std,0.391234,0.399467,0.229499,0.179164,0.400464,0.497067,0.237053,0.189219
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
75%,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,0.000000,0.000000
max,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000


In [46]:
tn = 0
fp = 0
fn = 0
tp = 0

for index, row in df_filtered.iterrows():
    image = row['Image_name']
    
    # Extrahiere die tatsächlichen Werte ab der zweiten Spalte
    actual_value_list = row[1:]
    if all(x == 0 for x in actual_value_list):
        actual_value = 0
    else:
        actual_value = 1

    # Finde die Vorhersagezeile für das Bild
    predicted_row = predictions_labels[predictions_labels["Image_name"] == image]
    
    if not predicted_row.empty:
        # Extrahiere die vorhergesagten Werte ab der zweiten Spalte
        predicted_value_list = predicted_row.iloc[0, 1:]
        
        if all(x == 0 for x in predicted_value_list):
            predicted_value = 0
        else:
            predicted_value = 1

        # Aktualisiere die Zähler basierend auf den tatsächlichen und vorhergesagten Werten
        if actual_value == 0 and predicted_value == 0:
            tn += 1
        elif actual_value == 0 and predicted_value == 1:
            fp += 1
        elif actual_value == 1 and predicted_value == 0:
            fn += 1
        elif actual_value == 1 and predicted_value == 1:
            tp += 1
    else:
        print(f"Keine Vorhersage für Bild: {image}")

print(f"TN: {tn}, FP: {fp}, FN: {fn}, TP: {tp}")


TN: 131, FP: 37, FN: 40, TP: 546


In [47]:
err = (fp+fn)/(tp+tn+fn+fp)
acc = (tp+tn)/(tp+tn+fn+fp)
f1 = (2*tp)/(2*tp+fp+fn)
precision = tp/(tp+fp)
recall = tp/(tp+fn)

print("We want to minimize false positives, so we want to maximize precision.")
print(f"Error rate: {round(err*100, 2)}\nAccuracy: {round(acc*100, 2)}\nF1: {round(f1,2)}\nPrecision: {round(precision,2)}\nRecall: {round(recall,2)}")

We want to minimize false positives, so we want to maximize precision.
Error rate: 10.21
Accuracy: 89.79
F1: 0.93
Precision: 0.94
Recall: 0.93
